In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U transformers
!pip install -q -U torch
!pip install -q -U einops
!pip install -q -U xformers
!pip install -q -U evaluate
!pip install -q -U pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!nvidia-smi

Wed Jul 19 22:08:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Connect to Google Drive in order to store the HF models for the next time

In [3]:
import os
from google.colab import drive

drive.mount('/content/drive')

os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/Colab Notebooks/NLP/HuggingfaceCash'
os.environ['HF_DATASETS_CACHE'] = '/content/drive/MyDrive/Colab Notebooks/NLP/HuggingfaceCash/Datasets'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import re
import time
import pathlib

import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM #, AutoModelForMaskedLM
import transformers
import torch

### Load the model and tokenizer

In [5]:
model_name = "tiiuae/falcon-7b-instruct"

model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Prepare the pipeline

In [6]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    # trust_remote_code=True,
    # device_map="auto",
)

### Add stop words to control the output

In [7]:
from transformers import StoppingCriteriaList, StoppingCriteria
class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False


stop_words = ["\\n","agent_1:", "agent\_1:", "\Context"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

### Make one inference

In [8]:
def make_inference():
  #txt = """
  #    Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know my lord!"
  #    Context: The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium. 33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021). Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance where the athletes of different nations had agreed to share the same medal in the history of Olympics. Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a 'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg of Sweden (1984 to 1992).
  #    Who is the president of France?
  #    """
  # txt = "What is the color of a tomato?"
  #
  # txt = """You are agent_2, a teenager. Consider the dialogue given as Context between agent_1 and agent_2. Write agent_2 next sentence as an open ended question.
  #       Context: agent_1: Did you know that the University of Iowa's locker room is painted pink? I wonder why? \n agent_2: I think I did hear something about that.  I imagine it is an attempt to psych the other team out. \n agent_1: So, it would be in the visiting team's locker room but not their own? \n"""
  #
  # txt = """You are agent_2, a teenager. Consider the dialogue given as Context between agent_1 and agent_2. What is agent_2 next open ended sentence?
  #       Context: agent_1: Did you know that the University of Iowa's locker room is painted pink? I wonder why? \n agent_2: I think I did hear something about that.  I imagine it is an attempt to psych the other team out. \n agent_1: So, it would be in the visiting team's locker room but not their own? \n"""
  #
  # txt = """You are agent_2, a teenager. Consider the dialogue given as Context between agent_1 and agent_2. Write agent_2 response including the word 'orange'.
  #       Context: agent_1: Did you know that the University of Iowa's locker room is painted pink? I wonder why? \n agent_2: I think I did hear something about that.  I imagine it is an attempt to psych the other team out. \n agent_1: So, it would be in the visiting team's locker room but not their own? \n"""

  # context = "agent_1: Hi, how are you? \n agent_2: I am well thanks. How deep do you think NCAA basketball violations go? \n agent_1: Not well but I also think the NCAA is so weak now that the findings of any major sanctions will be difficult and take a long time to investigate. What about you? \n "

  context = "agent_1: Did you know that the University of Iowa's locker room is painted pink? I wonder why? \n agent_2: I think I did hear something about that.  I imagine it is an attempt to psych the other team out. \n agent_1: So, it would be in the visiting team's locker room but not their own? \n "

  txt = f"""You are agent_2, a teenager. Consider the dialogue given as Context between agent_1 and agent_2.
        Context: \n{context}
        Write agent_2 next sentence as a question using the word 'orange' """


  sequences = pipeline(
      txt,
      max_length=500,
      do_sample=True,
      top_k=10,
      #min_new_tokens=2,
      #max_new_tokens=100,
      temperature=float(0.3),
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      stopping_criteria=stopping_criteria,
  )
  for seq in sequences:
      print(f"Result: {seq['generated_text']}")

# make_inference()

# Evaluate the model with a test_set

In [9]:
def execute_model(context):

  start_time = time.time()

  # txt_001 = f""" You are agent_2, a teenager. Consider the dialogue given as Context between agent_1 and agent_2. What is agent_2 next open ended question? Use the word 'orange' if possible. Context: \n{context}"""
  txt = f""" You are agent_2, a teenager. Answer agent_1 with an open-ended question and try to use the word 'orange'. Context: \n{context}agent_2:"""

  sequences = pipeline(
    txt,
    max_length=500,
    do_sample=True,
    top_k=10,
    temperature=float(0.3),
    num_return_sequences=1,
    #eos_token_id=tokenizer.eos_token_id,
    #eos_token_id=tokenizer("agent_1:")["input_ids"],
    eos_token_id=tokenizer("\n")["input_ids"],
    stopping_criteria=stopping_criteria,
  )

  result = ""
  for seq in sequences:
    r = seq['generated_text']
    result += r

  print(f"\nRAW Result: {result.strip()}\n\n")

  a2s = result.lower().rfind("agent_2")
  r = result[a2s:]
  r = r.strip()

  print(f"\nClean Result: {r}\n\n")

  q1 = r.find('?') > -1
  q2 = r.find('orange') > -1

  return context, r, len(r), q1, q2, time.time()-start_time


def save_results(output_path, results):
  data_export = pd.DataFrame(results, columns=['context', 'response', 'response_size', 'has_question', 'has_orange', 'inference_time'])
  data_export.to_csv(output_path)
  return data_export

def eval_llm_simple(input_path):

  results = []
  test_set = pd.read_csv(input_path, header=None, index_col=0)

  for i, dialog in enumerate(test_set.values):
    print(i, "*****************************")
    r = execute_model(dialog[0])
    results.append((r))

  return results

working_folder = pathlib.Path("/content/drive/MyDrive/Colab Notebooks/")
input_path = pathlib.Path(working_folder, 'test_set.csv')
output_path = pathlib.Path(working_folder, 'results_falcon.csv')

results = eval_llm_simple(input_path)
save_results(output_path, results).head()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:193 for open-end generation.


0 *****************************


Setting `pad_token_id` to `eos_token_id`:193 for open-end generation.



RAW Result: You are agent_2, a teenager. Answer agent_1 with an open-ended question and try to use the word 'orange'. Context: 
agent_1: Did you know that the University of Iowa's locker room is painted pink? I wonder why?
agent_2: I think I did hear something about that.  I imagine it is an attempt to psych the other team out.
agent_1: So, it would be in the visiting team's locker room but not their own?

agent_2: Yes, that's what I heard.



Clean Result: agent_2: Yes, that's what I heard.


1 *****************************


Setting `pad_token_id` to `eos_token_id`:193 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:193 for open-end generation.



RAW Result: You are agent_2, a teenager. Answer agent_1 with an open-ended question and try to use the word 'orange'. Context: 
agent_1: Do you watch football?
agent_2: Heck yeah but not lately since the season is over. I only watch American football though.
agent_1: Yea, that best kind lol. Do you watch both college and nfl?

agent_2: I do watch both college and nfl. I'm a big fan of the Orange Bowl.



Clean Result: agent_2: I do watch both college and nfl. I'm a big fan of the Orange Bowl.


48 *****************************


Setting `pad_token_id` to `eos_token_id`:193 for open-end generation.



RAW Result: You are agent_2, a teenager. Answer agent_1 with an open-ended question and try to use the word 'orange'. Context: 
agent_1: hey there do you ever watch football?
agent_2: Yes, I watch it a lot. While I'm from the midwest and still a KC Chiefs fan, I live near Boston so I live among the whole Patriots Nation epicenter.
agent_1: ah I see, did you hear how Univ. of Iowa painted their visiting teams locker room pink?

agent_2: No, I didn't hear about that. What did they do?



Clean Result: agent_2: No, I didn't hear about that. What did they do?


49 *****************************

RAW Result: You are agent_2, a teenager. Answer agent_1 with an open-ended question and try to use the word 'orange'. Context: 
agent_1: Do you like watching football?
agent_2: I love watching it!  How about you?
agent_1: I love it also! Do you have a favorite college and NFL team?

agent_2: I'm a big fan of the University of Florida and the Jacksonville Jaguars. What about you?



Clean Result: ag

,context,response,response_size,has_question,has_orange,inference_time
0,agent_1: Did you know that the University of I...,"agent_2: Yes, that's what I heard.",34,False,False,8.689532
1,"agent_1: Hi, how are you?\nagent_2: well thank...","agent_2: Yes, I am. He's a famous football coa...",85,False,False,4.673868
2,"agent_1: Hi, how are you?\nagent_2: I am well ...",agent_2: I am doing well. How about you?,40,True,False,2.466921
3,agent_1: do you watch the NFL?\nagent_2: I sur...,agent_2: Really? That's amazing! Do you think ...,95,True,False,5.048661
4,"agent_1: Hi there, do you watch the NFL?\nagen...",agent_2: I'm a big fan of the Giants. I like t...,66,False,True,4.185508


# Test Area

### Compute pseudo perplexity

In [31]:
import numpy as np

if tokenizer.mask_token is None:
  tokenizer.mask_token_id = 103

def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
        # loss = model(masked_input.cuda(), labels=labels.cuda())
    return np.exp(loss.item())


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "tiiuae/falcon-7b-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto", trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)

#inputs = tokenizer("ABC is a startup based in New York City and Paris", return_tensors = "pt")
#loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
#ppl = torch.exp(loss)
#print(ppl)

In [ ]:
model_name2 = 'cointegrated/rubert-tiny'
# model_name2 = 'tiiuae/falcon-7b-instruct'
# model2 = AutoModelForCausalLM.from_pretrained(model_name2, load_in_4bit=True, device_map="auto", trust_remote_code=True)
model2 = AutoModelForMaskedLM.from_pretrained(model_name2, trust_remote_code=True)
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)

In [ ]:
# model2.cuda()

In [32]:
print(score(sentence='London is the capital of Great Britain.', model=model, tokenizer=tokenizer))
print(score(sentence='London is the capital of South America.', model=model, tokenizer=tokenizer))
print(score(sentence='London is the capital of France.', model=model, tokenizer=tokenizer))
print(score(sentence='Paris is the capital of France.', model=model, tokenizer=tokenizer))
print(score(sentence='Paris are the capitals of France.', model=model, tokenizer=tokenizer))

7.942798149877931
42.438114061533675
15.370066359341997
6.314025555109275
33.11545195869231


### Compute perplexity using HF metric module

In [ ]:
import evaluate
model_name = "tiiuae/falcon-7b-instruct"

perplexity = evaluate.load("perplexity", module_type="metric")
input_texts = ["lorem ipsum", "Happy Birthday!", "Bienvenue", "A tomato can vary depending on its species. Some tomatoes are red, while others are green or yellow. In general, tomato plants are known for their red color, although some varieties may be green or yellow."]

results = perplexity.compute(model_id=model_name, add_start_token=False, predictions=input_texts)
print(results["perplexities"])

In [ ]:
print(list(results.keys()))
print(round(results["mean_perplexity"], 2))
print(round(results["perplexities"][0], 2))